# Santa 2025 – Local Metric Notebook

Local implementation of the evaluation metric for the **Santa 2025 – Christmas Tree Packing Challenge**.

This notebook:
1. Defines the Christmas tree polygon in local coordinates.
2. Provides a function to build polygons from (x, y, deg).
3. Loads a submission CSV (e.g. `sample_submission.csv`).
4. Computes the per-`n` score `s_n^2 / n` and total score.


In [ ]:
import math
from pathlib import Path

import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union

pd.set_option('display.float_format', '{:.12f}'.format)

PROJECT_ROOT = Path('.').resolve()
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'


In [ ]:
# Tree polygon in local coordinates; origin is center of top of trunk
TREE_TEMPLATE_VERTS = np.array([
    [0.0, 0.8],
    [0.25 / 2, 0.5],
    [0.25 / 4, 0.5],
    [0.4 / 2, 0.25],
    [0.4 / 4, 0.25],
    [0.7 / 2, 0.0],
    [0.15 / 2, 0.0],
    [0.15 / 2, -0.2],
    [-0.15 / 2, -0.2],
    [-0.15 / 2, 0.0],
    [-0.7 / 2, 0.0],
    [-0.4 / 4, 0.25],
    [-0.4 / 2, 0.25],
    [-0.25 / 4, 0.5],
    [-0.25 / 2, 0.5],
], dtype=float)

TREE_RADIUS = float(np.linalg.norm(TREE_TEMPLATE_VERTS, axis=1).max())
TREE_RADIUS


In [ ]:
def decode_val(s):
    """Decode competition-style 's0.123' strings to floats."""
    if isinstance(s, (float, int)):
        return float(s)
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def make_tree_polygon(x, y, angle_deg):
    """Construct a shapely Polygon for one tree."""
    theta = math.radians(angle_deg)
    c, s = math.cos(theta), math.sin(theta)
    rot = np.array([[c, -s], [s, c]], dtype=float)
    pts = TREE_TEMPLATE_VERTS @ rot.T
    pts[:, 0] += x
    pts[:, 1] += y
    return Polygon(pts)


In [ ]:
submission_path = DATA_RAW / 'sample_submission.csv'
if not submission_path.exists():
    raise FileNotFoundError(
        f'Missing {submission_path}; put sample_submission.csv in data/raw/'
    )

df_sub = pd.read_csv(submission_path)
df_sub.head()


In [ ]:
df = df_sub.copy()
df['n'] = df['id'].str.split('_').str[0].astype(int)
df['idx'] = df['id'].str.split('_').str[1].astype(int)

for col in ['x', 'y', 'deg']:
    df[col] = df[col].apply(decode_val)

df.head()


In [ ]:
def bounding_square_side_for_group(group: pd.DataFrame) -> float:
    polys = [
        make_tree_polygon(row['x'], row['y'], row['deg'])
        for _, row in group.iterrows()
    ]
    union = unary_union(polys)
    minx, miny, maxx, maxy = union.bounds
    width, height = maxx - minx, maxy - miny
    return max(width, height)

def metric_from_df(df: pd.DataFrame) -> pd.DataFrame:
    records = []
    for n, group in df.groupby('n'):
        side = bounding_square_side_for_group(group)
        records.append({
            'n': n,
            'side': side,
            'score_n': side * side / n,
            'count': len(group),
        })
    return pd.DataFrame(records).sort_values('n').reset_index(drop=True)


In [ ]:
score_table = metric_from_df(df)
display(score_table.head())
total_score = score_table['score_n'].sum()
print(f'Total score (local metric): {total_score:.12f}')
print('Distinct n:', score_table['n'].nunique())
print('Min n:', score_table['n'].min(), 'Max n:', score_table['n'].max())
print('Total trees:', score_table['count'].sum())


In [ ]:
import matplotlib.pyplot as plt

def plot_puzzle(df_all: pd.DataFrame, n: int, figsize=(6, 6)) -> None:
    group = df_all[df_all['n'] == n]
    polys = [
        make_tree_polygon(row['x'], row['y'], row['deg'])
        for _, row in group.iterrows()
    ]
    union = unary_union(polys)
    minx, miny, maxx, maxy = union.bounds
    side = max(maxx - minx, maxy - miny)
    fig, ax = plt.subplots(figsize=figsize)
    for poly in polys:
        xs, ys = poly.exterior.xy
        ax.fill(xs, ys, alpha=0.4)
        ax.plot(xs, ys, linewidth=0.8)
    ax.add_patch(plt.Rectangle((minx, miny), side, side,
                               fill=False, edgecolor='red',
                               linestyle='--', linewidth=2))
    pad = 0.5
    ax.set_xlim(minx - pad, minx + side + pad)
    ax.set_ylim(miny - pad, miny + side + pad)
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')
    ax.set_title(f'n={n}, side≈{side:.6f}')
    plt.show()

# Example: plot_puzzle(df, n=10)
